## Autoencoder experiment

1. Build an autoencoder and train it on train data (MIT's)
2. Save the encoded train data (MIT's)
3. Take encoder part from the AE and encode youtube data (the data that will work as "augmenter")
4. Apply some clustering or other algorithm to find similar segments in both encoded datasets (using some MIT videos as seeds)
5. Train NN on the MIT and YT data (those segments that are similar to MIT)
6. Validate the classifier (so you may need to leave some of the MIT and YT data out of the training)
7. Test on our own data

In [6]:
import os
import pickle
from scipy.io import wavfile
import numpy as np
from librosa import feature
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale


def extract(filename):
    rate, frames = wavfile.read(filename)
    window = 4096
    nfft = 64
    feat = []
    # len(frames)-window
    for i in range(0, len(frames) - window, window):
        # feat.append(np.array(frames[i:i + window]))
        feat.append(np.array(feature.mfcc(frames[i:i + window - 1],
                                   sr=rate,
                                   n_fft=nfft,
                                   hop_length=round(nfft / 2),
                                   fmax=8000)).flatten())
    feat = np.stack(feat)
    return feat
    
def extract_features(file_wet, file_dry):
    to_replace ="\\/"
    for char in to_replace:
        fw = file_wet.replace(char, "_")
        fd = file_dry.replace(char, "_")
    pickle_file = fw + "-" + fd + ".pkl"
    if os.path.exists(pickle_file):
        print("Using pickle file", pickle_file)
        with open(pickle_file, "rb") as f:
            features, labels = pickle.load(f)
        return features, labels
    features_wet = extract(file_wet)
    features_dry = extract(file_dry)
    labels_wet = np.ones(features_wet.shape[0])
    labels_dry = np.zeros(features_dry.shape[0])
    features = np.concatenate((features_wet, features_dry))
    labels = np.concatenate((labels_wet, labels_dry))
    labels = to_categorical(labels)
    features = minmax_scale(features)
    with open(pickle_file, "wb") as f:
        pickle.dump((features, labels), f, protocol=4)
    return features, labels
    
X, y = extract_features("yt_data/wet_mono.wav", "yt_data/dry_mono.wav")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

/media/adilkhan/HDD/Anaconda3/lib/python3.6/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


In [5]:
X_train.max()

1.0000000000000002

In [7]:
from keras.layers import Input, Dense
from keras.models import Model

# ae = models.Sequential()
#encoder
# d1 = Dense(1000, activation='tanh',input_shape=(X_train.shape[1:]))
# ae.add(d1)
# ae.add(BatchNormalization())
# #ae.add(Dropout(0.8))
# d2 = Dense(500, activation='tanh')
# ae.add(d2)
# #ae.add(Dropout(0.8))
# #decoder
# ae.add(Dense(d2.input_shape[1], activation='tanh'))
# ae.add(Dense(d1.input_shape[1], activation='tanh'))

inp = Input(shape=(X_train.shape[1:]))
encoded = Dense(1000, activation='relu')(inp)
encoded = Dense(500, activation='relu')(encoded)
encoded = Dense(200, activation='relu')(encoded)

decoded = Dense(500, activation='relu')(encoded)
decoded = Dense(1000, activation='relu')(decoded)
decoded = Dense(X_train.shape[1], activation='sigmoid')(decoded)
ae = Model(inp, decoded)
ae.compile(loss='mse',
optimizer='adadelta', metrics=['accuracy'])
ae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2560)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              2561000   
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_3 (Dense)              (None, 200)               100200    
_________________________________________________________________
dense_4 (Dense)              (None, 500)               100500    
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              501000    
_________________________________________________________________
dense_6 (Dense)              (None, 2560)              2562560   
Total para

In [8]:
ae.fit(X_train, X_train, epochs=100,
          batch_size=128,
          validation_data=(X_test, X_test))

Train on 467834 samples, validate on 230427 samples
Epoch 1/100
467834/467834 [==============================] - 38s - loss: 0.0082 - acc: 0.0134 - val_loss: 0.0067 - val_acc: 0.0070
Epoch 2/100
467834/467834 [==============================] - 32s - loss: 0.0066 - acc: 0.0094 - val_loss: 0.0066 - val_acc: 0.0143
Epoch 3/100
467834/467834 [==============================] - 32s - loss: 0.0066 - acc: 0.0213 - val_loss: 0.0066 - val_acc: 0.0226
Epoch 4/100
467834/467834 [==============================] - 32s - loss: 0.0066 - acc: 0.0281 - val_loss: 0.0066 - val_acc: 0.0266
Epoch 5/100
467834/467834 [==============================] - 34s - loss: 0.0065 - acc: 0.0234 - val_loss: 0.0065 - val_acc: 0.0178
Epoch 6/100
467834/467834 [==============================] - 32s - loss: 0.0064 - acc: 0.0173 - val_loss: 0.0064 - val_acc: 0.0177
Epoch 7/100
467834/467834 [==============================] - 32s - loss: 0.0064 - acc: 0.0166 - val_loss: 0.0064 - val_acc: 0.0171
Epoch 8/100
467834/467834 [====

467834/467834 [==============================] - 32s - loss: 0.0059 - acc: 0.0185 - val_loss: 0.0059 - val_acc: 0.0184
Epoch 63/100
467834/467834 [==============================] - 32s - loss: 0.0059 - acc: 0.0185 - val_loss: 0.0059 - val_acc: 0.0187
Epoch 64/100
467834/467834 [==============================] - 32s - loss: 0.0059 - acc: 0.0185 - val_loss: 0.0059 - val_acc: 0.0185
Epoch 65/100
467834/467834 [==============================] - 32s - loss: 0.0059 - acc: 0.0184 - val_loss: 0.0059 - val_acc: 0.0188
Epoch 66/100
467834/467834 [==============================] - 33s - loss: 0.0059 - acc: 0.0184 - val_loss: 0.0059 - val_acc: 0.0177
Epoch 67/100
467834/467834 [==============================] - 33s - loss: 0.0059 - acc: 0.0184 - val_loss: 0.0059 - val_acc: 0.0188
Epoch 68/100
467834/467834 [==============================] - 33s - loss: 0.0059 - acc: 0.0182 - val_loss: 0.0059 - val_acc: 0.0178
Epoch 69/100
467834/467834 [==============================] - 32s - loss: 0.0059 - acc: 0

In [10]:
ae.layers